In [1]:
# Minimal robust training script

import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc, f1_score, precision_score, recall_score
from sklearn.metrics import classification_report
import joblib
from pandas.api import types as ptypes
import numpy as np

In [2]:
# paths
merged_path = 'data/processed/train_merged.parquet'
feat_path = 'data/processed/train_features_day3_recomputed.parquet'
out_model = 'models/xgb_day4_baseline.joblib'

# 1) load
full = pd.read_parquet(merged_path)
feat = pd.read_parquet(feat_path)

print("full rows,cols:", full.shape, "feat rows,cols:", feat.shape)

full rows,cols: (590540, 435) feat rows,cols: (590540, 10)


In [3]:
# 2) attach recomputed features (safe: same order & length)

for c in feat.columns:
    if c not in full.columns:
        full[c] = feat[c].values

In [4]:
# 3) shortlist raw columns to include (important ones only)

base_cols = [
    'TransactionAmt', 'ProductCD',
    'card1', 'card2', 'card3', 'card4', 'card5', 'card6',
    'addr1', 'addr2',
    'DeviceInfo', 'DeviceType'
]

C_cols = [c for c in full.columns if str(c).startswith('C')]
D_cols = [c for c in full.columns if str(c).startswith('D')]
M_cols = [c for c in full.columns if str(c).startswith('M')]
V_cols = [c for c in full.columns if str(c).startswith('V')]
id_cols = [c for c in full.columns if str(c).startswith('id_')]

In [5]:
# engineered features

engineered = [c for c in feat.columns if c not in ['isFraud', 'dt']]

In [6]:
# build final feature list (dedupe, keep only existing)

feature_cols = []
for col_group in [base_cols, C_cols, D_cols, M_cols, V_cols, id_cols, engineered]:
    for c in col_group:
        if c in full.columns and c not in feature_cols:
            feature_cols.append(c)

print("Feature count:", len(feature_cols))

Feature count: 435


In [7]:
# 1) Convert object/categorical columns to integer codes (one-line safe)

for c in feature_cols:
    if c not in full.columns:
        continue

    # If it's object/string or categorical, convert to category then to codes
    if ptypes.is_object_dtype(full[c]) or ptypes.is_categorical_dtype(full[c]):
        full[c] = full[c].astype('category').cat.codes.astype('int32')
         # cat.codes uses -1 for NaN, replace that with a distinct sentinel so model doesn't confuse with a real code
        full[c].replace(-1, -999, inplace=True)

# 2) Now safe to fill numeric NaNs with -999 for the remaining columns
full[feature_cols] = full[feature_cols].fillna(-999)

/var/folders/0l/p2_v2ssd19s30xk4x88_7cy00000gn/T/ipykernel_76388/3442843167.py:8: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if ptypes.is_object_dtype(full[c]) or ptypes.is_categorical_dtype(full[c]):
/var/folders/0l/p2_v2ssd19s30xk4x88_7cy00000gn/T/ipykernel_76388/3442843167.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  full[c].replace(-1, -999, inplace=True)
/var/folders/0l/p2_v2ssd19s30xk4x88_7cy00000gn/T/ipykern

In [8]:
# Ensure dt exists and sort
if 'dt' not in full.columns:
    if 'TransactionDT' in full.columns:
        START_DATE = "2017-12-01"
        full['dt'] = pd.to_datetime(full['TransactionDT'], unit='s', origin=START_DATE)
    else:
        raise RuntimeError("No dt in full dataset.")
full = full.sort_values('dt').reset_index(drop=True)

In [9]:
full.to_parquet('data/processed/train_full.parquet')

In [ ]:
# Time split (70/15/15)


In [54]:
n =  len(full)
train_end = int(0.70 * n)
val_end = int(0.85 * n)

X_train = full.loc[:train_end, feature_cols]
y_train = full.loc[:train_end, 'isFraud'].astype(int)

X_val = full.loc[train_end:val_end, feature_cols]
y_val = full.loc[train_end:val_end, 'isFraud'].astype(int)

X_test = full.loc[val_end:, feature_cols]
y_test = full.loc[val_end:, 'isFraud'].astype(int)

print("Splits:", X_train.shape, X_val.shape, X_test.shape, "Fraud rate train/val/test:",
      y_train.mean(), y_val.mean(), y_test.mean())

Splits: (413379, 436) (88582, 436) (88581, 436) Fraud rate train/val/test: 0.0351686950715929 0.03434106251834459 0.03480430340592226


In [55]:
# class weight
pos = y_train.sum()
neg = len(y_train) - pos
scale_pos_weight = neg / (pos + 1e-9)
print("scale_pos_weight:", scale_pos_weight)

scale_pos_weight: 27.43437886916856


In [56]:
# train XGBoost baseline

model = XGBClassifier(
    n_estimators=2000,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.7,
    use_label_encoder=False,
    eval_metric='auc',
    tree_method='hist',
    scale_pos_weight=scale_pos_weight,
)

model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=100
)

[0]	validation_0-auc:0.82489


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/xgboost/training.py:183: UserWarning: [12:41:04] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[100]	validation_0-auc:0.88794
[200]	validation_0-auc:0.90092
[300]	validation_0-auc:0.90754
[400]	validation_0-auc:0.90913
[500]	validation_0-auc:0.91019
[600]	validation_0-auc:0.91180
[700]	validation_0-auc:0.91244
[800]	validation_0-auc:0.91374
[900]	validation_0-auc:0.91461
[1000]	validation_0-auc:0.91408
[1100]	validation_0-auc:0.91334
[1200]	validation_0-auc:0.91341
[1300]	validation_0-auc:0.91408
[1400]	validation_0-auc:0.91358
[1500]	validation_0-auc:0.91365
[1600]	validation_0-auc:0.91376
[1700]	validation_0-auc:0.91344
[1800]	validation_0-auc:0.91362
[1900]	validation_0-auc:0.91322
[1999]	validation_0-auc:0.91324


,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.7
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'auc'


In [58]:
# predict & evaluate (probabilities + threshold tuning)

y_val_proba = model.predict_proba(X_val)[:, 1]
y_test_proba = model.predict_proba(X_test)[:, 1]

roc_val = roc_auc_score(y_val, y_val_proba)
roc_test = roc_auc_score(y_test, y_test_proba)

In [62]:
# PR-AUC

prec, rec, thr = precision_recall_curve(y_val, y_val_proba)
pr_auc_val = auc(rec, prec)

prec_t, rec_t, thr_t = precision_recall_curve(y_test, y_test_proba)
pr_auc_test = auc(rec_t, prec_t)

In [63]:
# choose threshold on val maximizing f1 or a business point (here f1)
f1s = [(t, f1_score(y_val, (y_val_proba > t).astype(int))) for t in np.linspace(0.01,0.99,99)]
best_t, best_f1 = max(f1s, key=lambda x: x[1])
print("Best threshold on val (F1):", best_t, "F1:", best_f1)

Best threshold on val (F1): 0.72 F1: 0.5647840531561462


In [64]:
# compute test metrics at that threshold
y_test_pred = (y_test_proba > best_t).astype(int)
print("ROC Val/Test:", round(roc_val, 4), round(roc_test, 4))
print("PR-AUC Val/Test:", round(pr_auc_val,4), round(pr_auc_test,4))
print("Test classification report (threshold {:.3f}):".format(best_t))
print(classification_report(y_test, y_test_pred, digits=4))

ROC Val/Test: 0.9132 0.8815
PR-AUC Val/Test: 0.5851 0.5129
Test classification report (threshold 0.720):
              precision    recall  f1-score   support

           0     0.9796    0.9910    0.9853     85498
           1     0.6328    0.4288    0.5112      3083

    accuracy                         0.9715     88581
   macro avg     0.8062    0.7099    0.7483     88581
weighted avg     0.9676    0.9715    0.9688     88581



In [65]:
# recall at low FPRs (approx)
fprs = np.linspace(0.0001, 0.01, 10)
sorted_idx = np.argsort(-y_test_proba)
cum_preds = y_test_proba[sorted_idx]
cum_labels = y_test.values[sorted_idx]

In [66]:
# top-k capture (top 1% transactions)
k = int(0.01 * len(y_test))
topk_captured = cum_labels[:k].sum() / y_test.sum()
print("Top-1% fraud capture:", round(topk_captured,4))

Top-1% fraud capture: 0.2559


In [67]:
# feature importance
imp = pd.DataFrame({
    'feature': feature_cols,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False).head(50)
print("Top features:\n", imp.head(20))

Top features:
     feature  importance
307    V258    0.162544
119     V70    0.041551
250    V201    0.029599
140     V91    0.024049
267    V218    0.022498
343    V294    0.017623
366    V317    0.014292
313    V264    0.013801
371    V322    0.012555
9     addr2    0.011357
19       C8    0.010994
236    V187    0.010040
25      C14    0.009213
344    V295    0.007891
400   id_12    0.007606
7     card6    0.007420
306    V257    0.007050
16       C5    0.006859
336    V287    0.006828
329    V280    0.006550


In [69]:
# save model
joblib.dump(model, out_model)
print("Saved baseline model to:", out_model)

Saved baseline model to: models/xgb_day4_baseline.joblib


* Using full raw features + engineered velocity features boosted performance massively from ~0.74 ROC → 0.91 ROC (val).

* V-features dominate (V258, V257, V218, V201…). These carry the strongest fraud signals in IEEE-CIS.

* Test ROC is 0.882, lower than val due to temporal drift, which is expected in fraud data.

* PR-AUC ~0.51 on test indicates moderate precision under imbalance, still needs improvement.

* Top-1% capture = 25.7%, meaning the model catches ~26% of all fraud if you review only the top 1% risky transactions — room for major gains.

* Selected threshold (0.68) gives precision 0.60, recall 0.43 for fraud — usable but not ideal.

* The model is now stable and baseline-worthy. Further improvement must come from richer features, not parameter tuning.